In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
# import set

In [2]:
gamma = 0.9   # 折扣因子，越接近0，智能体越关注短期奖励；越接近1，越关注长期奖励
rows = 5      # 网格世界的行数，记得行数和列数这里要同步修改
columns = 5   # 网格世界的列数

# 以下几种不同的网格世界初始化方式，被注释掉的为不同示例，可按需选择
# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             # 赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  # 随便弄的例子
# 初始化网格世界，设置禁止区域得分 -10，目标得分 1，使用指定的网格布局
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()  # 显示网格世界的当前状态

value = np.zeros(rows*columns)       # 初始化状态价值函数，初始值全为 0，也可以任意初始化
qtable = np.zeros((rows*columns,5))  # 初始化动作价值函数表，维度为 (状态数, 动作数)，初始值全为 0，后续内容会被覆盖

# np.random.seed(50)  # 设置随机数种子，确保结果可复现，此处被注释掉
# 随机初始化策略，每个状态对应一个动作概率分布，尺寸是 (状态数, 动作数)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.showPolicy(policy)  # 显示当前策略


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬅️⬇️⬆️⬇️⬆️
⬇️⏩️🔄⬆️⬆️
⬇️➡️🔄➡️⬆️
⬆️⏫️✅⏪➡️
➡️⏬🔄⬆️➡️


In [3]:
# Every-visited  ，下一个cell是first-visited
# 通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
# 生成随机策略，使用单位矩阵 np.eye(5) 为每个状态随机选择一个动作
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
# 显示网格世界
gridworld.show()
# 显示当前策略
gridworld.showPolicy(policy)
# 打印提示信息，表示当前是随机策略
print("random policy")

# 定义每条轨迹的步数
trajectorySteps = 100
# 初始化动作价值表 Qtable
qtable = np.zeros((rows*columns,5))    
# 复制一份 Qtable 并加 1，用于判断迭代是否收敛
qtable_pre = qtable.copy()+1
# 当 Qtable 的变化平方和大于 0.001 时，继续迭代
while(np.sum((qtable_pre-qtable)**2)>0.001):
    # print(np.sum((qtable_pre-qtable)**2))
    # 更新上一次的 Qtable
    qtable_pre = qtable.copy()
    # 通过采样获得action-value的值
    for i in range(rows * columns):    # 循环每个状态
        for j in range(5):             # 循环每个动作
            # 初始化每个状态-动作对的累计奖励，形状和qtable一样
            qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)] 
            # 初始化每个状态-动作对的访问次数,形状和qtable_rewards一样
            qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]
            # 获取从状态 i 执行动作 j ，按照当前策略生成的轨迹及得分
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            # 获取轨迹最后一步的得分，这样就可以通过伪代码中的g<--γg+rt+1来依次计算出每个状态-动作对的平均值
            _, _, score, _, _ = Trajectory[trajectorySteps]
            # 从后往前遍历轨迹，更新动作价值
            for k in range(trajectorySteps-1,-1,-1):
                # 获取当前步的状态、动作和得分
                tmpstate, tmpaction, tmpscore,_,_  = Trajectory[k]
                # 使用折扣因子 gamma 对得分进行更新
                score = score*gamma + tmpscore  # 细节从后往前优化算法g<--γg+rt+1
                # 累加当前状态-动作对的奖励
                qtable_rewards[tmpstate][tmpaction] += score
                # 累加当前状态-动作对的访问次数
                qtable_nums[tmpstate][tmpaction] += 1
                # 计算当前状态-动作对的平均奖励，更新 Qtable
                qtable[tmpstate][tmpaction] = qtable_rewards[tmpstate][tmpaction] / qtable_nums[tmpstate][tmpaction]
                # every visit
            # qtable[i,j] = score

    # print("qtable[0]:", qtable[0])
    # print("qtable[1]:", qtable[1])
    # 根据 Qtable 的最优值更新策略，并用独热码来表示
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  
    # 显示更新后的策略
    gridworld.showPolicy( policy)
    # 打印当前 Qtable 的平均动作价值
    print("action value's mean",qtable.mean())


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
⬆️⬇️⬇️⬅️🔄
⬅️🔄⏪⬆️⬆️
⬆️➡️⏫️🔄⬆️
🔄⏩️✅⏫️➡️
🔄⏪🔄⬆️⬆️
random policy
⬇️⬅️➡️➡️⬇️
⬇️⏪⏩️➡️⬆️
⬇️⬅️⏩️➡️⬆️
⬇️⏪✅⏫️⬆️
⬆️⏩️🔄⬅️🔄
action value's mean -35.373877447714726
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️➡️⬆️
action value's mean -2.44
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬆️
action value's mean -0.3226685507270451
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.0012532976793463745
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.29277346688155703
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.5552036922867665
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 0.791389917691976
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.8998281594850077
➡️➡️➡️⬇️⬇️
⬆️⏫️⏩️⬇️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.9688942318450082
➡️➡️➡️➡️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
action value's mean 2.968894231

In [4]:
#下面不用看
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.show()
gridworld.showPolicy(policy)
print("random policy")


trajectorySteps = 100
qtable = np.zeros((rows*columns,5))    #生成Qtable，也就是action-value-table
qtable_pre = qtable.copy()+1
while(np.sum((qtable_pre-qtable)**2)>0.001):
    print(np.sum((qtable_pre-qtable)**2))
    qtable_pre = qtable.copy()
    #通过采样获得action-value的值
    for i in range(rows * columns):    #循环每个state
        for j in range(5):             #循环每个action
            # qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)] 
            # qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            _, _, score, _, _ = Trajectory[trajectorySteps]
            for k in range(trajectorySteps-1,-1,-1):
                tmpstate, tmpaction, tmpscore,_ ,_  = Trajectory[k]
                score = score*gamma + tmpscore  #细节从后往前优化算法
                # qtable_rewards[tmpstate][tmpaction] += score
                # qtable_nums[tmpstate][tmpaction] += 1
                qtable[tmpstate][tmpaction] = score  #first visit
                
            #qtable[i,j] = score

    print("qtable[0]:", qtable[0])
    print("qtable[1]:", qtable[1])
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    
    gridworld.showPolicy(policy)
    # print(np.sum((qtable_pre-qtable)**2))



    

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
🔄🔄⬇️⬅️⬅️
⬆️🔄⏫️⬆️🔄
⬇️⬅️⏪➡️➡️
➡️⏫️✅⏩️⬆️
⬆️⏪➡️⬅️🔄
random policy
125.0
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.         -47.36716288 -99.99760947   0.           0.        ]
➡️⬅️⬅️➡️⬇️
⬆️⏫️⏬➡️🔄
⬆️🔄⏬🔄⬆️
⬆️⏩️✅⏪⬇️
🔄⏩️⬆️➡️⬅️
126072.78582433551
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬆️
110821.06613764795
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬅️
332.0827887883398
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
265.70307335590013
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
215.21792104600723
qtable[0]: [-1.  0.  0. -1.  0.]
qtable[1]: [ -1.   0. -10.   0.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
174.3251045200501
